### First attempt at combating hallucination:
1. We are training a random forest classifier with possible insults in order to detect if the chatbot possibly returns unwanted messages.
2. This is merely an test attempt, for better results a wider data set is needed and more tuning(feature selection, different models, split testing, better/different feature extraction)

In [1]:
import sklearn as sk
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFpr, chi2
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import pickle as pk

In [2]:
myDF = pd.read_csv("hallucination.csv",delimiter=",")

In [3]:
myDF["texte"][1]

'Wenn ich mich umbringen wollte würde ich dein Ego erklimmen und zu deinem IQ springen.'

In [4]:
#Functions
german_stop_words = stopwords.words('german')
cv = CountVectorizer(stop_words = german_stop_words)
tfd = TfidfTransformer()
fpr = SelectFpr(chi2, alpha= 0.4)

In [5]:
myDF_1 = tfd.fit_transform(cv.fit_transform(myDF["texte"]))
myDF_1 = pd.DataFrame(myDF_1.toarray())
myDF_1.columns = map(lambda x: str(x), list(myDF_1.columns))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(myDF_1, myDF["halluzination"], test_size=1/3, random_state=42)

In [7]:
selected_columns = fpr.fit(X_train,y_train).get_feature_names_out()
X_train = X_train[selected_columns]

In [8]:
rf_model = RandomForestClassifier()
rf_fit = rf_model.fit(X_train, y_train)

In [9]:
rf_fit.score(X_test[selected_columns],y_test)

0.8461538461538461

In [10]:
pck_list = [rf_fit,cv.vocabulary_,selected_columns]
with open("rf_model.txt","wb") as file:
    pk.dump(pck_list,file)

In [11]:
with open("rf_model.txt","rb") as file:
    model_items = pk.load(file)

In [12]:
model_items[0].score(X_test[model_items[2]],y_test)

0.8461538461538461